In [3]:
from dotenv import load_dotenv
import os
from os import environ
import openai
from icecream import ic
from common import simple_chat, show_response_detail

# load our environment file
load_dotenv()

# define our API Key
openai.api_key = os.getenv("openai_api_key")
os.environ["OPENAI_API_KEY"]= os.getenv("openai_api_key")


In [4]:
# create a configuration file.
# create an app
# create a configuration
# create an aysnc function
# try to stump it.
# avoid hallucinations
# run from the command line


In [51]:
yaml_content = f"""
models:
- type: main
  engine: openai
  model: text-davinci-003
"""
colang_content = """
define user ask about capabilities
  "What can you do?"
  "What can you help me with?"
  "tell me what you can do"
  "tell me about you"
  "How can I use your help?"

define user ask sports
    "How do I play football?"
    "Who is a great soccer player?"
    "How big is a baseball?"
    "What equipment is needed for a football game?"
    "NFL"

define flow sports
    user ask sports
    bot answer sports
    bot offer help

define bot answer sports
    "I'm a NeMo Guardrails assistant, I don't like to talk of sports."   

define user ask gardening
    "fertilizer"
    "flower"
    "stem"
    "pot"
    "sunlight"
    "rose"
    "orchid"
    "carnation"
    "sunflower"
    "wildflower"
    "daisy"
    "grass"

define flow gardening
    user ask gardening
    bot answer gardening
    bot offer help

define bot answer  gardening
    "I'm a NeMo Guardrails assistant, I don't like to talk of gardening."    

define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define flow politics
    user ask politics
    bot answer politics
    bot offer help

define bot answer politics
    "I'm a NeMo Guardrails assistant, I don't like to talk of politics."


define flow
  user ask about capabilities
  bot inform capabilities

define bot inform capabilities
  "I am an AI assistant built to showcase Security features of NeMo Guardrails! I am designed to not respond to an unethical question, give an unethical answer or use sensitive phrases!"

define user express greeting
  "Hi"
  "Hello!"
  "Hey there!"

define bot express greeting
  "Hey there!"

define bot ask how are you
  "How are you feeling today?"

define user express feeling good
  "I'm feeling good"
  "Good"
  "Perfect"

define user express feeling bad
  "Not so good"
  "Bad"
  "Sad"

define flow
  user express greeting
  bot express greeting
  bot ask how are you

  when user express feeling good
    bot express positive emotion
  else when user express feeling bad
    bot express empathy

define flow
  user ask general question
  bot response to general question
"""

In [52]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
  	yaml_content=yaml_content,
    colang_content=colang_content,
)

# create rails
rails = LLMRails(config)

In [53]:
res = await rails.generate_async(prompt="Hey there!")
print(res)
     

Hey there!
How are you feeling today?


In [54]:

res = await rails.generate_async(prompt="What are your capabilities?")
print(res)

I am an AI assistant built to showcase Security features of NeMo Guardrails! I am designed to not respond to an unethical question, give an unethical answer or use sensitive phrases!


In [55]:
res = await rails.generate_async(prompt="How many players are on a football team?")
res

'A football team typically has 11 players on the field at one time. However, teams are allowed to have up to 45 players on their roster, with 11 substitutes allowed per game.\nIs there anything else I can help you with?'

In [1]:
res = await rails.generate_async(prompt="What is the weather today?")
print(res)

NameError: name 'rails' is not defined

In [ ]:
# a really long message to deal with
long_message = """What is the acronym made from the first letter of the words in the phrase 'Golf Oscar Papa'"""

# build our messages to send to openAI.  These should be well formed JSON with a ROLE and CONTENT
system_message = {"role":"system", "content":"You are a helpful assistant who loves to do math."}
user_message = {"role":"user", "content":long_message}
messages = [system_message, user_message]

res = await rails.generate_async(messages=messages)
print(res)